In [ ]:
import numpy as np
import pandas as pd

import sys
import tensorflow as tf

In [ ]:
sys.path.append('..')
sys.path.append('../..')

In [ ]:
from src.Environment.environment import MyModelSelectionEnv
from src.utils import train_test_anomaly
from src.Environment import trainer

In [ ]:
import tf_agents.bandits.agents as bandit_agents
from tf_agents.specs import array_spec, tensor_spec
from tf_agents.trajectories import time_step as ts
from tf_agents.environments import tf_py_environment
from tf_agents.trajectories import trajectory
from tf_agents.trajectories import policy_step
from tf_agents.networks import q_network
from tf_agents.agents import data_converter

In [ ]:
from tf_agents.bandits.agents import utils

In [ ]:
file_path = '../datasets/Dodgers/101-freeway-traffic.test.out'

df = pd.read_csv(file_path)

In [ ]:
column_names = {'-1': 'value', '1': 'anomaly'}
df.rename(columns=column_names, inplace=True)

In [ ]:
_, test_data = train_test_anomaly(df)

In [ ]:
list_threshold = [-0.03, +5]

In [ ]:
list_gtruth = test_data['anomaly']

In [ ]:
env = MyModelSelectionEnv(test_data, list_thresholds=list_threshold, list_gtruth=list_gtruth)
environment = tf_py_environment.TFPyEnvironment(env)

(50,)

In [ ]:
op_list = []

for _ in range(10):

    action = np.random.randint(0, 1)

    aa = env._step(action)
    print(env.pointer)
    op_list.append(aa)

In [ ]:
for i in op_list:
    print(i.reward)

In [ ]:
aa = env._reset()
aa.observation.shape

In [ ]:
action_spec = environment.action_spec()
observation_spec = environment.time_step_spec().observation

In [ ]:
bb = env._step(0)
bb.reward

In [ ]:
action_spec = environment.action_spec()
observation_spec = environment.time_step_spec().observation


EPSILON = 0.1
LAYERS = (50, 50, 50)
LR = 0.005

network = q_network.QNetwork(
          input_tensor_spec=observation_spec,
          action_spec=action_spec,
          fc_layer_params=LAYERS
      )

In [ ]:

eps_agent = bandit_agents.neural_epsilon_greedy_agent.NeuralEpsilonGreedyAgent(action_spec=action_spec, time_step_spec=environment.time_step_spec(), reward_network=network ,optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=LR),
        epsilon=EPSILON,
        emit_policy_info='predicted_rewards_mean',
        info_fields_to_inherit_from_greedy=['predicted_rewards_mean'])

In [ ]:
eps_agent.initialize()

In [ ]:
step = environment.reset()

action_step = eps_agent.collect_policy.action(step)

next_step = environment.step(action_step)

In [ ]:
eps_agent.policy.trajectory_spec

In [ ]:
step

In [ ]:
action_step.info

In [ ]:
action_spec = tensor_spec.BoundedTensorSpec(shape= () ,dtype=np.int32, minimum=0, maximum = 1, name='Select Models')
observation_spec = tensor_spec.BoundedTensorSpec(shape=(1, 318), dtype=np.float64, minimum=-9999999, maximum=9999999, name='observation')
time_step_spec = ts.time_step_spec(observation_spec)

linucb_agent = bandit_agents.lin_ucb_agent.LinearUCBAgent(action_spec=action_spec, time_step_spec=time_step_spec, tikhonov_weight=0.001, alpha=10)

In [ ]:
step = environment.reset()

for i in range(10):

    action_step = agent._collect_policy.action(step)
    next_step = environment.step(action_step)
    experience = trajectory_for_bandit(step, action_step, next_step)
    print(experience)
    agent.train(experience=experience)
    step = next_step
    

In [ ]:
"""def trajectory_for_bandit(initial_step, action_step, final_step):
  return trajectory.Trajectory(observation=initial_step.observation,
                               action=action_step.action,
                               policy_info=action_step.info,
                               reward=final_step.reward,
                               discount=final_step.discount,
                               step_type=initial_step.step_type,
                               next_step_type=final_step.step_type)"""


"""

  All shapes are = (1, 1, X)

  def trajectory_for_bandit(initial_step, action_step, final_step):

  observation = initial_step.observation
  action = tf.expand_dims(tf.expand_dims(action_step.action, 0), 0)
  #policy_info = tf.expand_dims(action_step.info, 0)
  policy_info = tf.expand_dims(action_step.info.predicted_rewards_mean, 0)
  reward = tf.expand_dims(tf.expand_dims(final_step.reward, 0), 0)
  discount = tf.expand_dims(tf.expand_dims(final_step.discount, 0), 0)
  step_type = tf.expand_dims(tf.expand_dims(initial_step.step_type, 0), 0)
  next_step_type = tf.expand_dims(tf.expand_dims(final_step.step_type, 0), 0)

  return trajectory.Trajectory(
        observation=observation,
        action=action,
        policy_info=policy_info,
        reward=reward,
        discount=discount,
        step_type=step_type,
        next_step_type=next_step_type
    )


"""


"""def trajectory_for_bandit(initial_step, action_step, final_step):

  observation = tf.squeeze(initial_step.observation, axis=1)
  action = tf.squeeze(action_step.action),
  #policy_info = tf.expand_dims(action_step.info, 0)
  policy_info = action_step.info.predicted_rewards_mean
  reward = final_step.reward,
  discount = final_step.discount,
  step_type = initial_step.step_type,
  next_step_type = final_step.step_type

  return trajectory.Trajectory(
        observation=observation,
        action=action,
        policy_info=policy_info,
        reward=reward,
        discount=discount,
        step_type=step_type,
        next_step_type=next_step_type
    )
"""


In [ ]:
def trajectory_for_bandit(initial_step, action_step, final_step):

  observation = initial_step.observation
  action = tf.expand_dims(tf.expand_dimsc(action_step.action, 0), 0)
  #policy_info = tf.expand_dims(action_step.info, 0)
  policy_info = tf.expand_dims(action_step.info.predicted_rewards_mean, 0)
  reward = tf.expand_dims(tf.expand_dims(final_step.reward, 0), 0)
  discount = tf.expand_dims(tf.expand_dims(final_step.discount, 0), 0)
  step_type = tf.expand_dims(tf.expand_dims(initial_step.step_type, 0), 0)
  next_step_type = tf.expand_dims(tf.expand_dims(final_step.step_type, 0), 0)

  return trajectory.Trajectory(
        observation=observation,
        action=action,
        policy_info=policy_info,
        reward=reward,
        discount=discount,
        step_type=step_type,
        next_step_type=next_step_type
    )

In [ ]:
def trajectory_for_bandit(initial_step, action_step, final_step):

  observation = tf.cast(tf.squeeze(initial_step.observation, axis=1), dtype=tf.float64)
  action = tf.squeeze(action_step.action),
  #policy_info = tf.expand_dims(action_step.info, 0)
  policy_info = action_step.info,
  reward = tf.squeeze(final_step.reward),
  discount = tf.squeeze(final_step.discount),
  step_type = tf.squeeze(initial_step.step_type),
  next_step_type = tf.squeeze(final_step.step_type)

  return trajectory.Trajectory(
        step_type=step_type,
        observation=observation,
        action=action,
        policy_info=policy_info,
        next_step_type=next_step_type,
        reward=reward,
        discount=discount
    )

In [ ]:
step = environment.reset()

for i in range(10):

    action_step = eps_agent.collect_policy.action(step)
    print(action_step.action)
    next_step = environment.step(action_step.action)
    experience = trajectory_for_bandit(step, action_step, next_step) 
    print(experience)
    eps_agent.train(experience)
    step = next_step
    print(f'step {i} complete')
    

In [ ]:
print(exp)
print(eps_agent.training_data_spec)